# RAGBackend Logic Summary

## Key Components:

1. **PDF Processing**
   - Extracts text from PDF documents
   - Splits text into smaller chunks for efficient processing

2. **Vector Store**
   - Creates embeddings for text chunks
   - Stores these embeddings for quick retrieval

3. **Question-Answering Chain**
   - Uses a language model (LLM) to generate answers
   - Retrieves relevant information from the vector store

## Process Flow:

1. **Initialization**
   - RAGBackend is initialized with an OpenAI API key
   - Sets up necessary components (vector store, QA chain)

2. **PDF Processing (`process_pdf` method)**
   - Extracts text from the uploaded PDF
   - Splits the text into manageable chunks
   - Creates embeddings for these chunks
   - Stores the embeddings in a vector store (FAISS)

3. **Question Answering (`ask_question` method)**
   - Receives a question from the user
   - Uses the vector store to retrieve relevant text chunks
   - Passes the question and relevant chunks to the LLM
   - Returns the generated answer

## Key Methods:

1. `_extract_text_from_pdf(pdf_path)`:
   - Uses PyPDF2 to extract text from each page of the PDF

2. `_split_text(text)`:
   - Uses LangChain's RecursiveCharacterTextSplitter to split text into chunks

3. `_create_vector_store(chunks)`:
   - Uses OpenAIEmbeddings to create embeddings
   - Stores these in a FAISS vector store

4. `_setup_qa_chain()`:
   - Sets up a RetrievalQA chain using the OpenAI language model and the vector store

5. `process_pdf(pdf_path)`:
   - Orchestrates the PDF processing workflow

6. `ask_question(question)`:
   - Handles the question-answering process

## Key Technologies:

- LangChain: For text splitting, embeddings, and QA chain setup
- OpenAI: For generating embeddings and language model responses
- FAISS: For efficient similarity search in the vector store
- PyPDF2: For PDF text extraction

This RAG system enhances traditional question-answering by grounding the LLM's responses in the specific content of the processed PDF, improving accuracy and relevance of the answers.

# Deploying RAG Application to SAP Kyma: Key Steps and Commands

1. **Build and Push Docker Image**
   ```bash
   docker build -t deepan13/rag-app:latest .
   docker push deepan13/rag-app:latest
   ```

2. **Create Kubernetes Secret for OpenAI API Key**
   ```bash
   kubectl create secret generic openai-credentials --from-env-file=.env
   ```

3. **Create Deployment and Service YAML**
   File: `rag-deployment.yaml`
   ```yaml
   apiVersion: apps/v1
   kind: Deployment
   metadata:
     name: rag-app
   spec:
     replicas: 1
     selector:
       matchLabels:
         app: rag-app
     template:
       metadata:
         labels:
           app: rag-app
       spec:
         containers:
         - name: rag-app
           image: deepan13/rag-app:latest
           ports:
           - containerPort: 8080
           envFrom:
           - secretRef:
               name: openai-credentials
   ---
   apiVersion: v1
   kind: Service
   metadata:
     name: rag-service
   spec:
     selector:
       app: rag-app
     ports:
       - protocol: TCP
         port: 80
         targetPort: 8080
   ```

4. **Apply Deployment and Service**
   ```bash
   kubectl apply -f rag-deployment.yaml
   ```

5. **Create API Rule YAML**
   File: `rag-api-rule.yaml`
   ```yaml
   apiVersion: gateway.kyma-project.io/v1beta1
   kind: APIRule
   metadata:
     name: rag-api-rule
   spec:
     gateway: kyma-gateway.kyma-system.svc.cluster.local
     host: rag-app
     service:
       name: rag-service
       port: 80
     rules:
       - accessStrategies:
           - handler: noop
         methods:
           - GET
           - POST
         path: /.*
   ```

6. **Apply API Rule**
   ```bash
   kubectl apply -f rag-api-rule.yaml
   ```

7. **Verify Deployment**
   ```bash
   kubectl get pods
   kubectl get services
   kubectl get apirules
   ```

8. **Get Application URL**
   ```bash
   kubectl get apirule rag-api-rule -o jsonpath='{.status.virtualService.hosts[0]}'
   ```

9. **Access Application**
   Open a web browser and navigate to:
   ```
   https://<hostname-from-previous-step>
   ```